# APRENDIZAJE AUTOMÁTICO
## MEMORIA Práctica 1: Auditoría de datos

Miguel Ibáñez González  
Alexandru Marius Platon  

Grupo: 14  
Fecha Entrega: 09/10/2024

## Apartado 1

## Obención de datos
Cargamos el archivo comprimido en un DataFrame. Definimos una lista llamada columns que contiene los nombres de las columnas.  
Usamos df.nunique() para calcular cuántos valores únicos existen en cada columna del DataFrame.  
Contamos el numero de fraudes detectados.  

In [ ]:
import pandas as pd


from os import path
DATA_DIR = "."
FILENAME = "datos_practica.txt.gz"
FILEPATH = path.join(DATA_DIR, FILENAME)
df = pd.read_csv(FILEPATH, sep='|', header=None)

columns = [
    "Fecha Transaccion", "Hora Transaccion", "Cliente Id", "Perfil Cliente", "Segmento", "Ip", 
    "Modo Acceso", "Id Sesion", "Importe", "Tipo Mensaje", "Canal", "Fecha Sesion", "Hora Sesion", 
    "Medio Autenticacion", "Tipo Transaccion", "Entidad", "Oficina Origen", "Cuenta Origen", 
    "Entidad Destino", "Oficina Destino", "Cuenta Destino", "Tipo Firma", "Tipo Cuenta Origen", 
    "Pais Destino", "Fecha Alta Canal", "Fecha Activacion Canal", "Fecha Nac Titu Cta Cargo", 
    "Fecha Alta Cta Cargo", "Pais Ip", "Latitud", "Longitud", "Browser", "Browser Version", 
    "Os", "Os Version", "Profesion Cliente", "Sector Cliente", "Segmento Cliente", "Indicador Fraude"
]
df.columns = columns
unique_values_counts = df.nunique()
fraude_count = (df["Indicador Fraude"] == 1).sum()
print(f"Número de fraudes detectados:  {fraude_count}")
print(unique_values_counts)

En la celda anterior vemos que pandas guarda los valores como enteros en la mayoría de los casos

## Filtrado de Columnas
Realizamos un filtrado básico de campos, eliminando aquellos que solo contienen un valor ya que no dan información adicional, y los campos que no ofrecen información de especial interés.  

In [ ]:
filtered_columns = unique_values_counts[unique_values_counts != 1].index
filtered_df = df[filtered_columns]
columns_to_drop = [
    "Cliente Id", "Ip", "Id Sesion", "Oficina Origen", "Cuenta Origen", 
    "Entidad Destino", "Oficina Destino", "Cuenta Destino", "Fecha Alta Canal", "Fecha Activacion Canal", 
    "Fecha Nac Titu Cta Cargo", "Fecha Alta Cta Cargo", "Latitud", "Longitud",
]

filtered_df = filtered_df.drop(columns=columns_to_drop)
print(filtered_df.head)

## Campos con un valor único
- Segmento:  Vacio  
- Browser:  Navegador desconocido  
- Browser Version:  Vacio  
- Os:  Os desconocido  
- Os Version:  Vacio  
- Profesion Cliente: Vacio  

## Análisis Adicional: Frecuencia en campos categóricos con muchos valores
Para las columnas descartadas, contamos los valores únicos y se imprimieron para dar contexto a su eliminación.  

In [ ]:
for columna in columns_to_drop:
    n_unicos = len(df[columna].dropna().unique())
    print(f'Número de valores únicos en {columna}: {n_unicos}')

## Visualización de datos númericos
Creamos los histogramas y boxplots para visualizar la distribución de ciertos campos:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


plt.figure(figsize=(8, 4))
plt.hist(filtered_df["Indicador Fraude"], edgecolor='purple', color='pink')
plt.title(f'Frecuencia de {"Indicador Fraude"}')
unique_values = sorted(filtered_df["Indicador Fraude"].unique())
plt.xticks(unique_values, rotation=45)  
plt.tight_layout()
plt.show()

q75, q25 = np.percentile(filtered_df["Importe"], [75, 25])
iqr = q75 - q25 
upper_limit = q75 + 1.5 * iqr
filtered_data = filtered_df[filtered_df["Importe"] <= upper_limit]

plt.figure(figsize=(8, 4))
plt.boxplot(filtered_data["Importe"], boxprops=dict(color='black'), 
            medianprops=dict(color='red'), whiskerprops=dict(color='black'), showmeans=True)

plt.title('Boxplot de Importe')
plt.xticks([1], ['Importe'])
plt.tight_layout()
plt.show()





En el primer gráfico, 1 indica que hay fraude mientras que 0 indica lo contrario. Dado que solo hay 82 fraudes mientras que hay casi 50.000 que no lo son, la barra de los fraudes casi no se aprecia.  

En la segunda gráfica se muestra un boxplot dividido por percentiles donde se puede ver la mediana de los datos marcada por la línea roja. Se ve que generalmente las transacciones suelen ser de valores inferiores a 100.

## Visualización de datos no númericos
Eliminamos las columnas numéricas del DataFrame filtrado para enfocarse en los campos categóricos.  
Creamos un gráfico de barras para las 40 categorías más frecuentes de cada campo categórico, visualizando su frecuencia.  


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

numerical_fields = [
  "Indicador Fraude", "Fecha Transaccion", "Hora Transaccion", "Fecha Sesion", "Hora Sesion", "Importe"
]

filtered_df = filtered_df.drop(columns=numerical_fields)

filtered_df
for field in filtered_df:
    plt.figure(figsize=(8, 4))
    #sns.histplot(x=df[field])
    value_counts = filtered_df[field].value_counts().nlargest(40)
    
    plt.bar(value_counts.index, value_counts.values, edgecolor='purple', color='pink')
    plt.title(f'Frecuencia de {field}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


## Análisis de Campos de Fecha
Convertimos "Fecha Transacción" y "Fecha Sesión" a formato de fecha y se generaron series temporales para visualizar la frecuencia de transacciones y sesiones a lo largo del tiempo.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df['Fecha Transaccion'] = pd.to_datetime(df['Fecha Transaccion'], format='%Y%m%d')
df['Fecha Sesion'] = pd.to_datetime(df['Fecha Sesion'], format='%Y%m%d')
date_fields = [
    'Fecha Transaccion', 'Fecha Sesion',
]

for field in date_fields:
    plt.figure(figsize=(10, 6))
    
    time_series_data = df[field].value_counts().sort_index() 
    
    plt.plot(time_series_data.index, time_series_data.values, marker='o', linestyle='-', color='black')
    
    plt.title(f'Serie Temporal de {field}')
    plt.xlabel('Fechas')
    plt.ylabel('Frecuencia')
    plt.xticks(rotation=45)
    plt.tight_layout()
 
    plt.show()


Vemos que las gráficas para Fecha Sesion y Fecha transacción son prácticamente idénticas.  
Notamos también bajadas periódicas en el número de transacciones con frecuencia de una semana aproximadamente. Estas bajadas coinciden con fechas de fines de semana. 

## Análisis de Campos de Hora
Para el caso de las horas, tratamos los datos como cadenas de texto y seleccionamos solo los carácteres correspondientes a las horas (los dos primeros).   
Posteriormente los mostramos como enteros para seguir un orden cronológico

In [ ]:
hour_fields = [
    "Hora Transaccion", "Hora Sesion"
]

df['Hora Transaccion'] = df['Hora Transaccion'].astype(str).str.zfill(6).str[:2].astype(int)
df['Hora Sesion'] = df['Hora Sesion'].astype(str).str.zfill(6).str[:2].astype(int)

for field in hour_fields:
    plt.figure(figsize=(10, 5))
    plt.hist(df[field], bins=23, edgecolor='purple', color='pink') 
    plt.title(f'Frecuencia de {field}')
    plt.xlabel('Hora')
    plt.ylabel('Frecuencia')
    plt.xticks(range(24))  
    plt.xlim(-0.5, 23.5)
    plt.tight_layout()
    plt.show()


Se puede aprecias como entre las 00 y las 7 casi no hay transacciones (lo que es normal dado que es por la noche).

# Apartado 2

## Par de valores: Importe - Hora Transacción

- Normalización: Utilizamos StandardScaler para normalizar las columnas de "Importe" para que tengan media 0 y desviación estándar 1. Esto permite que los datos se comparen en la misma escala. Al ser las horas un tipo de dato categórico, no es necesario normalizarlas.
- Visualización: Usamos un gráfico de dispersión (scatter plot) para visualizar las transacciones genuinas (azul) y fraudulentas (rojo) según el importe y la hora, mostrando si hay algún patrón que distinga las operaciones fraudulentas de las genuinas.  

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

new_df = df.copy()

selected_fields = ["Importe", "Hora Transaccion", "Indicador Fraude"]
selected_df = new_df[selected_fields]

scaler = StandardScaler()
selected_df.loc[:, ['Importe']] = scaler.fit_transform(selected_df[['Importe']]).astype(int)

plt.figure(figsize=(10, 6))
plt.scatter(selected_df[selected_df['Indicador Fraude'] == 0]['Importe'], 
            selected_df[selected_df['Indicador Fraude'] == 0]['Hora Transaccion'], 
            label="Genuino", alpha=0.6, c="blue", marker='o')
plt.scatter(selected_df[selected_df['Indicador Fraude'] == 1]['Importe'], 
            selected_df[selected_df['Indicador Fraude'] == 1]['Hora Transaccion'], 
            label="Fraude", alpha=0.6, c="red", marker='x')

plt.title("Importe vs. Hora de Transacción - Visualización en 2D (Normalizado)")
plt.xlabel("Importe Normalizado")
plt.ylabel("Hora de Transacción Normalizada")
plt.xticks(rotation=45)
plt.grid()
plt.legend()
plt.show()




Vemos que los fraudes se producen a partir de las 08h hasta aproximadamente las 22h. Por otro lado, la gran mayoria de fraudes tienen valores cercanos a 0 como importes. En el gráfico de dispersión no se observan puntos claros en los que haya distinciones.

## Par de valores: Importe - Segmento Cliente

- Normalización: Normalizamos el importe de la misma forma que en el apartado anterior. Segmento cliente es una campo categórico, por lo que no tendría mucho sentido normalizarlo.
- Visualización: Creamos un gráfico de dispersión que muestra la relación entre "Importe" y "Segmento Cliente", diferenciando entre transacciones genuinas y fraudulentas.  

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

new_df = df.copy()

selected_fields = ["Importe", "Segmento Cliente", "Indicador Fraude"]
selected_df = new_df[selected_fields]

scaler = StandardScaler()

selected_df.loc[:, 'Importe'] = scaler.fit_transform(selected_df[['Importe']]).astype(int)

plt.figure(figsize=(10, 6))
plt.scatter(selected_df[selected_df['Indicador Fraude'] == 0]['Importe'], 
            selected_df[selected_df['Indicador Fraude'] == 0]['Segmento Cliente'], 
            label="Genuino", alpha=0.6, c="blue", marker='o')
plt.scatter(selected_df[selected_df['Indicador Fraude'] == 1]['Importe'], 
            selected_df[selected_df['Indicador Fraude'] == 1]['Segmento Cliente'], 
            label="Fraude", alpha=0.6, c="red", marker='x')

plt.title("Importe vs. Segmento Cliente - Visualización en 2D (Normalizado)")
plt.xlabel("Importe Normalizado")
plt.ylabel("Segmento Cliente Codificado")
plt.xticks(rotation=45)
plt.grid()
plt.legend()
plt.show()


En el gráfico se puede distinguir algunos perfiles de clientes que son mas propensos a fraude que otros, entre el 31 y el 33 se producen prácticamente todos los fraudes. En este grafico de dispersión tampoco se observan puntos claros en los que haya distinciones.

## Par de valores: Tipo de Mensaje - Medio Autenticación
- Normalización: En este caso se trata de dos campos categóricos, por lo que no se normalizan.  
- Visualización: Creamos un gráfico de dispersión para comparar las transacciones genuinas y fraudulentas según el "Tipo Mensaje" y el "Medio de Autenticación".

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

new_df = df.copy()

selected_fields = ["Medio Autenticacion", "Tipo Mensaje", "Indicador Fraude"]
selected_df = new_df[selected_fields]


plt.figure(figsize=(10, 6))
plt.scatter(selected_df[selected_df['Indicador Fraude'] == 0]['Tipo Mensaje'], 
            selected_df[selected_df['Indicador Fraude'] == 0]['Medio Autenticacion'], 
            label="Genuino", alpha=0.6, c="blue", marker='o')	
plt.scatter(selected_df[selected_df['Indicador Fraude'] == 1]['Tipo Mensaje'], 
            selected_df[selected_df['Indicador Fraude'] == 1]['Medio Autenticacion'], 
            label="Fraude", alpha=0.6, c="red", marker='x')

plt.title("Tipo Mensaje vs. Medio Autenticacion - Visualización en 2D (Normalizado)")
plt.xlabel("Tipo Mensaje")
plt.ylabel("Medio Autenticacion")
plt.xticks(rotation=45)
plt.grid()
plt.legend()
plt.show()


En este grafico se puede observar como todos los fraudes se producen con el Tipo de Mensaje PB, y como los con los Medios de Autenticación S2 y D3 no se producen fraudes. En este grafico de dispersión tampoco se observan puntos claros en los que haya distinciones.

## Par de valores: Tipo Firma - Medio Autenticación
- Normalización: Se trata nuevamente de dos campos categóricos, por lo que no se aplica normalización.
- Visualización: Creamos un scatter plot para ver la relación entre "Tipo Firma" y "Medio Autenticación" para transacciones genuinas y fraudulentas.  

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

new_df = df.copy()


selected_fields = ["Medio Autenticacion", "Tipo Firma", "Indicador Fraude"]
selected_df = new_df[selected_fields]


plt.figure(figsize=(10, 6))
plt.scatter(selected_df[selected_df['Indicador Fraude'] == 0]['Tipo Firma'], 
            selected_df[selected_df['Indicador Fraude'] == 0]['Medio Autenticacion'], 
            label="Genuino", alpha=0.6, c="blue", marker='o')
plt.scatter(selected_df[selected_df['Indicador Fraude'] == 1]['Tipo Firma'], 
            selected_df[selected_df['Indicador Fraude'] == 1]['Medio Autenticacion'], 
            label="Fraude", alpha=0.6, c="red", marker='x')

plt.title("Tipo Firma vs. Medio Autenticacion - Visualización en 2D (Normalizado)")
plt.xlabel("Tipo Firma")
plt.ylabel("Medio Autenticacion")
plt.xticks(rotation=45)
plt.grid()
plt.legend()
plt.show()


En este grafico de dispersión se observa como las transacciones fraudulentas se realizan con los Tipos de Firma T3, T6, S1 y T1. En este grafico de dispersión tampoco se observan puntos claros en los que haya distinciones.

Después de analizar todos estos pares de valores, vemos que no hay una clara distinción explicita entre las diferentes clases en función de los campos, pero sí observamos algunos campos que pueden indicar mayor probabilidad de fraude que otros.

# Apartado 3

## Calculamos la probabilidad a priori de que una transacción sea fraudulenta

In [ ]:
new_df = df.copy()
total_cases = len(new_df)
fraud_cases = len(new_df[new_df['Indicador Fraude'] == 1])
fraud_probability = fraud_cases / total_cases
print(f'La probabilidad a priori de ser fraude es: {fraud_probability:.4f}')

## Clasificador DummyClassifier(strategy='stratified')

- En `X` tomamos todos los valores menos la columna **"Indicador Fraude"**, que contiene la información de fraude y será la variable objetivo.
- `y` es la variable objetivo, que contiene la columna **"Indicador Fraude"**.
- `train_test_split()` divide los datos en conjuntos de entrenamiento (`X_train`, `y_train`) y de prueba (`X_test`, `y_test_stratified`).
- Usamos el **20%** del total de los datos para la prueba (`test_size=0.2`), mientras que el **80%** se utiliza para el entrenamiento.
- `random_state=42` garantiza que siempre se obtenga la misma división al ejecutar el código.
- El clasificador hace predicciones respetando la proporción de clases en los datos.
- Se entrena el modelo Dummy con los datos de entrenamiento (`X_train` y `y_train`).
- `DummyClassifier` predice las etiquetas de fraude (`y_pred_stratified`) en los datos de prueba (`X_test`).



In [28]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split

X = new_df.drop(columns=['Indicador Fraude'])
y = new_df['Indicador Fraude']  

X_train, X_test, y_train, y_test_stratified = train_test_split(X, y, test_size=0.2, random_state=42)

dummy_clf_stratified = DummyClassifier(strategy='stratified')

dummy_clf_stratified.fit(X_train, y_train)

y_pred_stratified = dummy_clf_stratified.predict(X_test)



## Clasificador DummyClassifier(strategy='most_frequent')
El clasificador usa la estrategia `'most_frequent'`, por lo que siempre predice la clase más frecuente en los datos de entrenamiento.

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score, recall_score


X = new_df.drop(columns=['Indicador Fraude'])  
y = new_df['Indicador Fraude'] 


X_train, X_test, y_train, y_test_most_frequent = train_test_split(X, y, test_size=0.2, random_state=42)


dummy_clf_most_frequent = DummyClassifier(strategy='most_frequent', random_state=42)


dummy_clf_most_frequent.fit(X_train, y_train)


y_pred_most_frequent = dummy_clf_most_frequent.predict(X_test)


## Calcular accuracy, sensitivity (TPR), specificity (TNR), balanced accuracy y la matriz de confusión para los dos casos
- Matriz de Confusión: muestra los verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos para las predicciones del modelo.
- Calculamos la matriz de confusión para los dos modelos (`cm_stratified` para el modelo estratificado y `cm_most_frequent` para el modelo de la clase más frecuente).
- Con `accuracy_score()` medimos el porcentaje de predicciones correctas (cuántos ejemplos fueron bien clasificados).
- Con `recall_score()` mostramos qué proporción de los fraudes reales fueron correctamente identificados por el modelo.
- La **especificidad (TNR)** se calcula dividiendo los verdaderos negativos entre la suma de verdaderos negativos y falsos positivos. Esto indica qué proporción de las transacciones no fraudulentas fueron correctamente identificadas.
- Con `balanced_accuracy_scoreo` medimos la precisión balanceada promedia, la sensibilidad (recall) y la especificidad.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, balanced_accuracy_score

cm_stratified = confusion_matrix(y_test_stratified, y_pred_stratified)
cm_most_frequent = confusion_matrix(y_test_most_frequent, y_pred_most_frequent)

print("Métricas del Modelo Estratificado:")
print(f"Precisión: {accuracy_score(y_test_stratified, y_pred_stratified):.4f}")
print(f"Sensibilidad (TPR): {recall_score(y_test_stratified, y_pred_stratified):.4f}")
print(f"Especificidad (TNR): {cm_stratified[0, 0] / (cm_stratified[0, 0] + cm_stratified[0, 1]):.4f}")
print(f"Precisión Balanceada: {balanced_accuracy_score(y_test_stratified, y_pred_stratified):.4f}")
print(f"Matriz de Confusión:\n{cm_stratified}\n")

print("Métricas del Modelo de Frecuencia Más Alta:")
print(f"Precisión: {accuracy_score(y_test_most_frequent, y_pred_most_frequent):.4f}")
print(f"Sensibilidad (TPR): {recall_score(y_test_most_frequent, y_pred_most_frequent):.4f}")
print(f"Especificidad (TNR): {cm_most_frequent[0, 0] / (cm_most_frequent[0, 0] + cm_most_frequent[0, 1]):.4f}")
print(f"Precisión Balanceada: {balanced_accuracy_score(y_test_most_frequent, y_pred_most_frequent):.4f}")
print(f"Matriz de Confusión:\n{cm_most_frequent}\n")


### Resultados

#### 1. Modelo Estratificado (`DummyClassifier(strategy='stratified')`)

- **Precisión**: **0.9959**
  - El modelo tiene una precisión del 99.59% (muy alta). Pero esto es porque la mayoría de las transacciones son **no fraudulentas**. Dado que el modelo no tiene poder predictivo, simplemente está replicando la proporción de clases en los datos.

- **Sensibilidad (True Positive Rate - TPR)**: **0.0000**
  - La sensibilidad es **0%**, lo que significa que el modelo **no identificó ningún fraude** correctamente. Este resultado tiene sentido, teniendo en cuenta la baja probabilidad de que una transacción sea fraudulenta y que el modelo solamente trata de igualar esa proporción de forma aleatoria.

- **Especificidad (True Negative Rate - TNR)**: **0.9979**
  - La especificidad es **muy alta (99.79%)**, indicando que el modelo es excelente para identificar transacciones **no fraudulentas**. Pero esto ocurre porque la mayoría de las transacciones pertenecen a esta clase de no fraudes.

- **Precisión Balanceada**: **0.4990**
  - La precisión balanceada es casi del 50%, lo que indica un rendimiento pobre en la clasificación de ambas clases (fraude y no fraude). Este valor destaca la diferencia entre la alta especificidad y la nula sensibilidad.

- **Matriz de Confusión**: [9995 21] --> (No fraudes predichos como no fraudes, 9995 correctos y 21 incorrectos) [ 20 0]] --> (Fraudes predichos como fraudes, 0 correctos y 20 incorrectos)


#### 2. Modelo de Frecuencia Más Alta (`DummyClassifier(strategy='most_frequent')`)

- **Precisión**: **0.9980**
- La precisión es aún mayor (99.80%) que en el modelo estratificado, pero, al igual que antes, esta alta precisión indica que el modelo predice la clase mayoritaria (no fraude) para la mayoría de los casos.

- **Sensibilidad (TPR)**: **0.0000**
- La sensibilidad es **0%**, lo que significa que este modelo también **no identifica ningún fraude**. Siempre predice la clase mayoritaria (no fraude), ignorando completamente los fraudes.

- **Especificidad (TNR)**: **1.0000**
- La especificidad es **100%**, lo que significa que el modelo clasifica todas las transacciones no fraudulentas de manera perfecta (no comete errores ya que sólo predice la clase no fraude).

- **Precisión Balanceada**: **0.5000**
- La precisión balanceada es **50%**, lo que refleja un desequilibrio completo: el modelo es perfecto en la clase no fraudulenta pero completamente ineficaz en la clase fraudulenta.

- **Matriz de Confusión**: [[10016 0] --> (No fraudes predichos como no fraudes, 10016 correctos y 0 incorrectos) [ 20 0]] --> (Fraudes predichos como fraudes, 0 correctos y 20 incorrectos)
